In [1]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [2]:
datapath = "../../../deeplearning/TensorFlow2.0_ResNet/dataset/"
# datapath = "../../../deeplearning/covidDataset/"
print(os.path.exists(datapath))
image_dims = (224, 224)

True


In [3]:
if (os.path.exists(datapath)==True):
    train_datagen = ImageDataGenerator(rescale = 1./255)
    test_datagen = ImageDataGenerator(rescale = 1./255)
    validation_datagen = ImageDataGenerator(rescale = 1./255)
    train_generator = train_datagen.flow_from_directory(os.path.join(datapath, "train"),
                                                 target_size = image_dims,
                                                batch_size = 16,
                                                 class_mode = 'categorical',
                                                color_mode = 'rgb')
    test_generator = test_datagen.flow_from_directory(os.path.join(datapath, "test"), 
                                                                 target_size = image_dims,
                                                                  batch_size = 16,
                                                                 class_mode = 'categorical', 
                                                                 color_mode = 'rgb')
    validation_generator = validation_datagen.flow_from_directory(os.path.join(datapath, "valid"), 
                                                         target_size = image_dims,
                                                          batch_size = 16,
                                                         class_mode = 'categorical', 
                                                         color_mode = 'rgb')

Found 1488 images belonging to 2 classes.
Found 495 images belonging to 2 classes.
Found 498 images belonging to 2 classes.


In [4]:
# train_generator, test_generator, validation_generator = get_imagedatagenerator(datapath)
# x_train, y_train = next(train_generator)
# x_test, y_test = next(test_generator)
# x_valid, y_valid = next(validation_generator)
# n = 6
# plt.figure(figsize=(20, 4))
# for i in range(n):
#     ax = plt.subplot(1, n, i+1)
#     ax.set(xlabel="label - {}".format(y_train[i]))
#     img = x_train[i]
#     ax.imshow(img, cmap='gray')
# plt.show()

In [5]:
import numpy as np

In [6]:
xtrain , ytrain = next(train_generator)

In [7]:
print(ytrain[1])

[0. 1.]


In [8]:
import tensorflow as tf
# import keras

In [9]:
vgg_model = tf.keras.applications.VGG19(weights='imagenet',
                                      include_top=False,
                                        input_shape=(224, 224, 3))
vgg_model.trainable=False 

x = vgg_model.layers[-1].output
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1000, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(500, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(200, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
vgg_output = tf.keras.layers.Dense(2, activation='softmax')(x)

vgg = tf.keras.models.Model(vgg_model.input, vgg_output)
vgg._name='vgg19'
# vgg.summary()

In [10]:
vgg.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=[tf.keras.metrics.BinaryAccuracy()]
)

In [11]:
history = vgg.fit(train_generator,
                    epochs=1)

93/93 [==============================] - 114s 1s/step - loss: 1.7932 - binary_accuracy: 0.5659


In [12]:
# test_loss, test_acc = vgg.evaluate(test_generator)

In [13]:
dense169_model = tf.keras.applications.DenseNet169(weights='imagenet',
                                      include_top=False,
                                        input_shape=(224, 224, 3)
                                      )
# dense169_model.summary()
dense169_model.trainable=False 

y = dense169_model.layers[-1].output
y = tf.keras.layers.Flatten()(y)
y = tf.keras.layers.Dropout(0.5)(y)
y = tf.keras.layers.Dense(1000, activation='relu')(y)
y = tf.keras.layers.Dropout(0.5)(y)
densenet_1_output = tf.keras.layers.Dense(2, activation='softmax')(y)

densenet_1 = tf.keras.models.Model(dense169_model.input, densenet_1_output)
densenet_1._name='densenet_1'
# densenet_1.summary()

In [14]:
densenet_1.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=[tf.keras.metrics.BinaryAccuracy()]
)

In [15]:
densenet_1_history = densenet_1.fit(train_generator,
                    epochs=1,
                    batch_size=16)

93/93 [==============================] - 116s 1s/step - loss: 5.0573 - binary_accuracy: 0.7809


In [16]:
# test_loss, test_acc = densenet_1.evaluate(test_generator)

In [17]:
dense169_model2 = tf.keras.applications.DenseNet169(weights='imagenet',
                                      include_top=False,
                                        input_shape=(224, 224, 3)
                                      )
# dense169_model.summary()
dense169_model2.trainable=False 
# since the model is already loaded we load in again
z = dense169_model2.layers[-1].output
z = tf.keras.layers.Flatten()(z)
z = tf.keras.layers.Dropout(0.5)(z)
z = tf.keras.layers.Dense(1000, activation='relu')(z)
z = tf.keras.layers.Dropout(0.5)(z)
z = tf.keras.layers.Dense(500, activation='relu')(z)
z = tf.keras.layers.Dropout(0.5)(z)
z = tf.keras.layers.Dense(200, activation='relu')(z)
z = tf.keras.layers.Dropout(0.5)(z)
densenet_2_output = tf.keras.layers.Dense(2, activation='softmax')(z)

densenet_2 = tf.keras.models.Model(dense169_model2.input, densenet_2_output)
densenet_2._name='densenet_2'
# densenet_2.summary()

In [18]:
densenet_2.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=[tf.keras.metrics.BinaryAccuracy()]
)

In [19]:
densenet_2_history = densenet_2.fit(train_generator,
                    epochs=1,
                    batch_size = 32)

93/93 [==============================] - 105s 1s/step - loss: 7.7802 - binary_accuracy: 0.6156


In [20]:
vgg.trainable = False
densenet_1.trainable = False
densenet_2.trainable = False
models = [vgg, densenet_1, densenet_2]
# to avoid conflicts with naming
for layer in models[0].layers:
    layer._name = layer.name + str('_0')
for layer in models[1].layers:
    layer._name = layer.name + str('_1')
for layer in models[2].layers:
    layer._name = layer.name + str('_2')

In [21]:
average = tf.keras.layers.Average()([vgg.output, densenet_1.output, densenet_2.output])
dense = tf.keras.layers.Dense(1, activation='sigmoid')(average)

ensemble = tf.keras.Model([vgg.input, densenet_1.input, densenet_2.input], dense)

In [22]:
ensemble.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=[tf.keras.metrics.BinaryAccuracy()]
)

In [23]:
# class DataGenerator(tf.keras.utils.Sequence):
#     'Generates data for Keras'
#     def __init__(self, paths, labels, batch_size=16, dim=(224, 224), n_channels=3,
#                  n_classes=2, shuffle=True):
#         'Initialization'
#         self.dim = dim
#         self.batch_size = batch_size
#         self.labels = labels
#         self.paths = paths
#         self.n_channels = n_channels
#         self.n_classes = n_classes
#         self.shuffle = shuffle
#         self.on_epoch_end()

#     def __len__(self):
#         'Denotes the number of batches per epoch'
#         return int(len(self.paths) / self.batch_size)

#     def __getitem__(self, index):
#         'Generate one batch of data'
#         # Generate indexes of the batch
#         indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

#         # Find list of IDs
#         list_paths = [self.paths[k] for k in indexes]
#         y = [self.labels[k] for k in indexes]
#         y = np.array(y)

#         # Generate data
#         X = self.__data_generation(list_paths)

#         return [X,X,X],y

#     def on_epoch_end(self):
#         'Updates indexes after each epoch'
#         self.indexes = np.arange(len(self.paths))
#         if self.shuffle == True:
#             np.random.shuffle(self.indexes)

#     def __data_generation(self, list_paths):
#         'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
#         # Initialization
#         X = np.empty((self.batch_size, *self.dim, self.n_channels))

#         # Generate data
#         for i, path in enumerate(list_paths):           
#             image = tf.keras.preprocessing.image.load_img(path)
#             image_arr = tf.keras.preprocessing.image.img_to_array(image)
#             image_arr = tf.image.resize(image_arr,(self.dim[0], self.dim[1])).numpy()
#             X[i,] = image_arr/255
#         return X

In [24]:
# import pathlib

# x_train = list(pathlib.Path(os.path.join(datapath, "train")).glob("*/*"))
# print(len(x_train))
# y_train = [pathlib.Path(path).parent.name for path in x_train]
# print(len(y_train))

# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# le.fit(y_train)
# labels = le.transform(y_train)
# print(y_train[1], " ", labels[1])
# print(y_train[1000], " ", labels[1000])

In [25]:
# train_gen = DataGenerator(x_train, labels)

In [26]:
# x, y = train_gen[0]

In [27]:
# print(len(x))

# print(y.shape)

### the below approach of multiple image inputs to a ```model.fit()``` using ```ImageDataGenerator```was used from [github issues](https://github.com/keras-team/keras/issues/8130)

In [28]:
def generate_generator_multiple(generator,dir1, batch_size, img_height,img_width):
    genX1 = generator.flow_from_directory(dir1,
                                          target_size = (img_height,img_width),
                                          class_mode = 'categorical',
                                          color_mode='rgb',
                                          batch_size = batch_size,
                                          shuffle=False, 
                                          seed=7)
    
    genX2 = generator.flow_from_directory(dir1,
                                          target_size = (img_height,img_width),
                                          class_mode = 'categorical',
                                          color_mode = 'rgb',
                                          batch_size = batch_size,
                                          shuffle=False, 
                                          seed=7)
    genX3 = generator.flow_from_directory(dir1,
                                          target_size = (img_height,img_width),
                                          class_mode = 'categorical',
                                          color_mode = 'rgb',
                                          batch_size = batch_size,
                                          shuffle=False, 
                                          seed=7)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            X3i = genX3.next()
            yield [X1i[0], X2i[0], X3i[0]], X3i[1]  #Yield both images and their mutual label
            
input_gen = ImageDataGenerator(rescale=1.0/255)
input_generator = generate_generator_multiple(generator=input_gen,
                                             dir1=os.path.join(datapath, "train"),
                                             batch_size=16,
                                              img_height=224,
                                              img_width=224)

In [29]:
# x, y = next(input_generator)

In [30]:
# def generator(samples, labels, batch_size=16):
#     """
#     Yields the next training batch.
#     Suppose `samples` is an array [[image1_filename,label1], [image2_filename,label2],...].
#     """
#     num_samples = len(samples)
#     while True: # Loop forever so the generator never terminates
# #         shuffle(samples)
 
#         # Get index to start each batch: [0, batch_size, 2*batch_size, ..., max multiple of batch_size &lt;= num_samples]
#         for offset in range(0, num_samples, batch_size):
#             # Get the samples you'll use in this batch
#             batch_samples = samples[offset:offset+batch_size]
#             batch_labels = labels[offset:offset+batch_size]
#             # Initialise X_train and y_train arrays for this batch
#             X_train = []
#             y_train = []
 
#             # For each example
#             for batch_sample in batch_samples:
#                 # Load image (X)
# #                 filename = './common_filepath/'+batch_sample[0]
#                 image = tf.keras.preprocessing.image.load_img(batch_sample)
#                 image_arr = tf.keras.preprocessing.image.img_to_array(image)
#                 image_arr = tf.image.resize(image_arr,(224, 224)).numpy()
#                 image = image_arr / 255
#                 # Read label (y)
# #                 y = batch_sample[1]
#                 # Add example to arrays
#                 X_train.append(image)
#             for label in batch_labels:
#                 y = label
#                 y_train.append(y)
 
#             # Make sure they're numpy arrays (as opposed to lists)
#             X_train = np.array(X_train)
#             y_train = np.array(y_train)
 
#             # The generator-y part: yield the next training batch            
#             yield [X_train, X_train, X_train], y_train

In [31]:
# train = generator(x_train, labels)

In [32]:
# x, y = train.__next__()
# print(x.shape)
# print(y.shape)

In [39]:
ensemble_history = ensemble.fit(input_generator,
                                epochs=1,
                                steps_per_epoch=93)

93/93 [==============================] - 320s 3s/step - loss: 0.6939 - binary_accuracy: 0.5000


## resources
### [Using generators in Python to train machine learning models](http://www.jessicayung.com/using-generators-in-python-to-train-machine-learning-models/)

### [Write your own Custom Data Generator for TensorFlow Keras](https://medium.com/analytics-vidhya/write-your-own-custom-data-generator-for-tensorflow-keras-1252b64e41c3)

### [A detailed example of how to use data generators with Keras](https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly)

### [Github Issue :  how to use fit_generator with multiple image inputs #8130 ](https://github.com/keras-team/keras/issues/8130)